In [1]:
import pandas as pd
import pycountry

# 读取数据文件
data_path = "data.csv"
df = pd.read_csv(data_path)

# 创建 3-letter 到 2-letter 国家代码的映射
country_mapping = {}
for country in pycountry.countries:
    # 有些国家可能没有alpha_3属性
    if hasattr(country, 'alpha_3') and hasattr(country, 'alpha_2'):
        country_mapping[country.alpha_3] = country.alpha_2

# 为某些特殊情况添加映射
special_cases = {
    'XKX': 'XK',  # Kosovo
    'ESH': 'EH',  # Western Sahara
    'MMR': 'MM',  # Myanmar/Burma
    'TWN': 'TW',  # Taiwan
    'VAT': 'VA',  # Vatican City
    'PRK': 'KP',  # North Korea
    'KOR': 'KR',  # South Korea
    'MKD': 'MK',  # North Macedonia
    'SWZ': 'SZ',  # Eswatini (formerly Swaziland)
}

# 更新映射字典
country_mapping.update(special_cases)


In [2]:
# 将RegionLabel列从3-letter转换为2-letter代码
df['RegionLabel'] = df['RegionLabel'].map(lambda x: country_mapping.get(x, x))

# 检查是否有未能成功映射的国家代码
unmapped = df[~df['RegionLabel'].isin(list(country_mapping.values()) + list(country_mapping.keys()))]
if len(unmapped) > 0:
    print("无法映射的国家代码:")
    print(unmapped[['Region', 'RegionLabel']])


In [3]:
# 保存更新后的数据
df.to_csv(data_path, index=False)
print("数据已更新并保存")

# 显示前几行查看结果
df.head()


数据已更新并保存


,Region,RegionLabel,Color,ColorGroup,Population (people)
0,Afghanistan,AF,#fbb4ae,3,38326027
1,Albania,AL,#fbb4ae,3,2872130
2,Algeria,DZ,#b3cde3,1,43090987
3,Andorra,AD,#ccebc5,5,77020
4,Angola,AO,#decbe4,0,32898997


In [55]:
# 读取rank202405文件
rank_path = '../202405/handle/rank202405.csv'
rank_df = pd.read_csv(rank_path)

# 按国家分组，计算cone字段的总和以及每个国家的记录数量
country_stats = rank_df.groupby('country').agg({
    'cone': 'sum',
    'country': 'count'
}).rename(columns={'country': 'num'}).reset_index()

# 显示结果的前几行
print("Country statistics:")
print(country_stats.head())


Country statistics:
  country  cone  num
0      AD     1    1
1      AE   878   68
2      AF   141   42
3      AG  1214    7
4      AI     4    3


In [56]:
# 读取data.csv文件(之前可能已经修改过)
data_df = pd.read_csv(data_path)

# 检查data.csv是否已有cone和num列，如果没有则添加
if 'cone' not in data_df.columns:
    data_df['cone'] = 0
if 'num' not in data_df.columns:
    data_df['num'] = 0

# 直接使用country匹配RegionLabel
for index, row in country_stats.iterrows():
    country_name = row['country']
    
    # 直接查找对应的RegionLabel行
    matched_rows = data_df[data_df['RegionLabel'] == country_name]
    if len(matched_rows) > 0:
        data_df.loc[data_df['RegionLabel'] == country_name, 'cone'] = row['cone']
        data_df.loc[data_df['RegionLabel'] == country_name, 'num'] = row['num']
    else:
        print(f"未找到匹配的国家: {country_name}")

# 保存更新后的data.csv
data_df.to_csv(data_path, index=False)
print("数据已更新，添加了cone和num字段")

# 显示更新后的前几行数据
data_df.head()


未找到匹配的国家: AI
未找到匹配的国家: AS
未找到匹配的国家: AW
未找到匹配的国家: AX
未找到匹配的国家: BL
未找到匹配的国家: BM
未找到匹配的国家: BQ
未找到匹配的国家: CK
未找到匹配的国家: CW
未找到匹配的国家: EU
未找到匹配的国家: FK
未找到匹配的国家: FO
未找到匹配的国家: GF
未找到匹配的国家: GG
未找到匹配的国家: GI
未找到匹配的国家: GP
未找到匹配的国家: GU
未找到匹配的国家: HK
未找到匹配的国家: IM
未找到匹配的国家: IO
未找到匹配的国家: JE
未找到匹配的国家: KY
未找到匹配的国家: MF
未找到匹配的国家: MO
未找到匹配的国家: MP
未找到匹配的国家: MQ
未找到匹配的国家: MS
未找到匹配的国家: NC
未找到匹配的国家: NF
未找到匹配的国家: NU
未找到匹配的国家: PF
未找到匹配的国家: PM
未找到匹配的国家: PR
未找到匹配的国家: PS
未找到匹配的国家: RE
未找到匹配的国家: SX
未找到匹配的国家: TC
未找到匹配的国家: TK
未找到匹配的国家: VG
未找到匹配的国家: VI
未找到匹配的国家: WF
未找到匹配的国家: YT
数据已更新，添加了cone和num字段


,Region,RegionLabel,Color,ColorGroup,Population (people),cone,num
0,Afghanistan,AF,#fbb4ae,3,38326027,141,42
1,Albania,AL,#fbb4ae,3,2872130,220,63
2,Algeria,DZ,#b3cde3,1,43090987,16,8
3,Andorra,AD,#ccebc5,5,77020,1,1
4,Angola,AO,#decbe4,0,32898997,2573,37


In [57]:
# 检查有多少国家成功匹配，有多少无法匹配
matched_countries = data_df[data_df['num'] > 0]['Region'].tolist()
print(f"成功匹配的国家数量: {len(matched_countries)}")
print(f"未匹配的国家数量: {len(data_df) - len(matched_countries)}")

# 查看未匹配的国家列表
unmatched_countries = data_df[data_df['num'] == 0]['Region'].tolist()
print("部分未匹配的国家:")
print(unmatched_countries[:10] if len(unmatched_countries) > 10 else unmatched_countries)


成功匹配的国家数量: 194
未匹配的国家数量: 4
部分未匹配的国家:
['Korea, North', 'Kosovo', 'Namibia', 'Western Sahara']


In [58]:
# 读取关系数据文件
rel_path = '../202405/handle/rel202405.csv'
rel_df = pd.read_csv(rel_path)

# 创建ID到国家的映射字典
id_to_country = dict(zip(rank_df['id'], rank_df['country']))

# 添加source_country和target_country列
rel_df['source_country'] = rel_df['source'].map(id_to_country)
rel_df['target_country'] = rel_df['target'].map(id_to_country)

# 检查是否有无法映射的ID
unmapped_sources = rel_df[rel_df['source_country'].isna()]['source'].unique()
unmapped_targets = rel_df[rel_df['target_country'].isna()]['target'].unique()

if len(unmapped_sources) > 0 or len(unmapped_targets) > 0:
    print(f"无法映射的source ID数量: {len(unmapped_sources)}")
    print(f"无法映射的target ID数量: {len(unmapped_targets)}")

# 删除无法映射的记录
rel_df = rel_df.dropna(subset=['source_country', 'target_country'])


无法映射的source ID数量: 1
无法映射的target ID数量: 6


In [59]:
# 统计每个国家的对等连接数量(relation=0)
peer_connections = rel_df[rel_df['relation'] == 0].groupby('source_country').size().reset_index(name='peer_connections')

# 统计每个国家作为提供商(provider)的连接数量(relation=-1)
p2c_connections = rel_df[rel_df['relation'] == -1].groupby('source_country').size().reset_index(name='p2c_connections')

# 统计每个国家作为客户(customer)的连接数量(relation=-1，但是从target角度)
c2p_connections = rel_df[rel_df['relation'] == -1].groupby('target_country').size().reset_index(name='c2p_connections')
c2p_connections = c2p_connections.rename(columns={'target_country': 'source_country'})

# 合并所有统计数据
country_stats = pd.DataFrame({'country': rank_df['country'].unique()})
country_stats = country_stats.merge(peer_connections, left_on='country', right_on='source_country', how='left')
country_stats = country_stats.merge(p2c_connections, left_on='country', right_on='source_country', how='left')
country_stats = country_stats.merge(c2p_connections, left_on='country', right_on='source_country', how='left')

# 填充缺失值为0
country_stats = country_stats.fillna(0)

# 删除多余的列
if 'source_country_x' in country_stats.columns:
    country_stats = country_stats.drop(['source_country_x', 'source_country_y', 'source_country'], axis=1, errors='ignore')
elif 'source_country' in country_stats.columns:
    country_stats = country_stats.drop('source_country', axis=1, errors='ignore')

# 将统计数据转换为整数
country_stats['peer_connections'] = country_stats['peer_connections'].astype(int)
country_stats['p2c_connections'] = country_stats['p2c_connections'].astype(int)
country_stats['c2p_connections'] = country_stats['c2p_connections'].astype(int)

# 计算总连接数
country_stats['total_connections'] = country_stats['peer_connections'] + country_stats['p2c_connections'] + country_stats['c2p_connections']

# 显示统计结果的前几行
country_stats.head(10)


,country,peer_connections,p2c_connections,c2p_connections,total_connections
0,US,63431,43749,34150,141330
1,SE,4864,3629,906,9399
2,IT,17758,1911,1827,21496
3,FR,16642,1554,2360,20556
4,EU,139,195,57,391
5,ES,3001,1278,1520,5799
6,NL,27809,2331,2134,32274
7,RU,62748,7754,8100,78602
8,CO,485,425,384,1294
9,HK,6726,1533,1610,9869


In [60]:
# 计算全球总计
total_peer = country_stats['peer_connections'].sum()
total_p2c = country_stats['p2c_connections'].sum()
total_c2p = country_stats['c2p_connections'].sum()
total_connections = country_stats['total_connections'].sum()

print(f"全球总对等连接数: {total_peer}")
print(f"全球总P2C连接数: {total_p2c}")
print(f"全球总C2P连接数: {total_c2p}")
print(f"全球总连接数: {total_connections}")


全球总对等连接数: 520435
全球总P2C连接数: 129615
全球总C2P连接数: 129615
全球总连接数: 779665


In [61]:


# 添加新列到data_df
if 'peer_connections' not in data_df.columns:
    data_df['peer_connections'] = 0
if 'p2c_connections' not in data_df.columns:
    data_df['p2c_connections'] = 0
if 'c2p_connections' not in data_df.columns:
    data_df['c2p_connections'] = 0
if 'total_connections' not in data_df.columns:
    data_df['total_connections'] = 0

# 直接使用country匹配RegionLabel更新data_df中的连接数据
matched_count = 0
for idx, row in country_stats.iterrows():
    country = row['country']
    # 直接使用RegionLabel匹配
    matched_rows = data_df[data_df['RegionLabel'] == country]
    if len(matched_rows) > 0:
        data_df.loc[data_df['RegionLabel'] == country, 'peer_connections'] = row['peer_connections']
        data_df.loc[data_df['RegionLabel'] == country, 'p2c_connections'] = row['p2c_connections']
        data_df.loc[data_df['RegionLabel'] == country, 'c2p_connections'] = row['c2p_connections']
        data_df.loc[data_df['RegionLabel'] == country, 'total_connections'] = row['total_connections']
        matched_count += 1
    else:
        print(f"未匹配: {country}")

print(f"成功匹配的国家数量: {matched_count}")
print(f"未匹配的国家数量: {len(country_stats) - matched_count}")


未匹配: EU
未匹配: HK
未匹配: VG
未匹配: PR
未匹配: GI
未匹配: PS
未匹配: NC
未匹配: GP
未匹配: BM
未匹配: CW
未匹配: GU
未匹配: JE
未匹配: MO
未匹配: MQ
未匹配: PF
未匹配: IM
未匹配: KY
未匹配: SX
未匹配: AW
未匹配: BQ
未匹配: MF
未匹配: GF
未匹配: FO
未匹配: RE
未匹配: VI
未匹配: 0
未匹配: BL
未匹配: MP
未匹配: AI
未匹配: NU
未匹配: NF
未匹配: GG
未匹配: MS
未匹配: TK
未匹配: CK
未匹配: AS
未匹配: PM
未匹配: TC
未匹配: WF
未匹配: AX
未匹配: FK
未匹配: IO
未匹配: YT
成功匹配的国家数量: 194
未匹配的国家数量: 43


In [62]:
# 保存更新后的data.csv文件
# data_df去掉Population (people)这一列，然后上面新增的列添加后面追加一个空格(size)
data_df = data_df.drop(columns=['Population (people)'], errors='ignore')
data_df.to_csv("new_data202405.csv", index=False)
print("数据已更新并保存")

# 显示更新后的前几行数据
data_df.head()


数据已更新并保存


,Region,RegionLabel,Color,ColorGroup,cone,num,peer_connections,p2c_connections,c2p_connections,total_connections
0,Afghanistan,AF,#fbb4ae,3,141,42,18,57,76,151
1,Albania,AL,#fbb4ae,3,220,63,40,87,137,264
2,Algeria,DZ,#b3cde3,1,16,8,7,8,17,32
3,Andorra,AD,#ccebc5,5,1,1,33,0,6,39
4,Angola,AO,#decbe4,0,2573,37,605,169,70,844
